<a href="https://colab.research.google.com/github/sonmiannkim/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [21]:
from  pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameBasics").getOrCreate()

In [22]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33605939| RGYFDX8QXKEIR|B007KO2MLO|     328837464|           Big Maria|Digital_Ebook_Pur...|          4|            0|          0|   N|                N|              Quirky|Elmore Leonard me...| 2013-09-09|
|         US|   34058393|R13CBGTMNV9R8Z|B005FLODDE|     764276359|The Woman Who Was...|Digital_Ebook_Pur

In [23]:
# Create the vine_table where total votes over 20 
vine_tv_over_twenty_df = df.filter("total_votes>20").select(["review_id", "star_rating", "helpful_votes", "total_votes","vine","verified_purchase"])
vine_tv_over_twenty_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R39ESX43X1SA5T|          1|           36|         47|   N|                Y|
|R31QRZ3JAMPBB4|          1|           16|         26|   N|                Y|
|R23FKBEXURC3SN|          5|           28|         36|   N|                N|
| R4IAIDV5EE84W|          1|            8|         21|   N|                Y|
|R26MB2DA7ROT4P|          1|            2|         21|   N|                Y|
| RTRVYV0GWUTCK|          5|           18|         25|   N|                Y|
|R3QLZAI37SNOOC|          2|           30|         31|   N|                Y|
|R1QYZU3RS4CY4F|          5|           66|         76|   N|                N|
|R2UR5OLN1BU87J|          2|           21|         23|   N|                Y|
| RYBZ6CTBOGI0H|          1|            3|         84|   N|     

In [24]:
# Retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
vine_tv_percent_df = vine_tv_over_twenty_df.filter("helpful_votes/total_votes >= 0.5")
vine_tv_percent_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R39ESX43X1SA5T|          1|           36|         47|   N|                Y|
|R31QRZ3JAMPBB4|          1|           16|         26|   N|                Y|
|R23FKBEXURC3SN|          5|           28|         36|   N|                N|
| RTRVYV0GWUTCK|          5|           18|         25|   N|                Y|
|R3QLZAI37SNOOC|          2|           30|         31|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [25]:
#total_filtered = 60647
vine_tv_percent_df.count()

60647

In [26]:
vine_paid_df = vine_tv_percent_df.filter(vine_tv_percent_df["vine"] == "Y")
vine_paid_df.show(5)

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [27]:
# total number of vine paid 0 , there fore further calculation also will be all zero
total_number_of_vine_paid = vine_paid_df.count()
print ("The total number of vine reviews = " + str(total_number_of_vine_paid)) 

The total number of vine reviews = 0


In [28]:
vine_not_paid_df = vine_tv_percent_df.filter(vine_tv_percent_df["vine"] == "N")
vine_not_paid_df.show(5)


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R39ESX43X1SA5T|          1|           36|         47|   N|                Y|
|R31QRZ3JAMPBB4|          1|           16|         26|   N|                Y|
|R23FKBEXURC3SN|          5|           28|         36|   N|                N|
| RTRVYV0GWUTCK|          5|           18|         25|   N|                Y|
|R3QLZAI37SNOOC|          2|           30|         31|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [29]:
#total number of vine not paid 60647
total_number_of_vine_not_paid = vine_not_paid_df.count()
print ("The total number of vine not paid reviews = " + str(total_number_of_vine_not_paid))

The total number of vine not paid reviews = 60647


In [30]:
#total number of vine paid - unfortunately I chose the category where there is no paid Vine counts after filter, it should be zero
total_5star_Vine_paid = vine_paid_df.filter("star_rating = 5").count()
print ("The number of 5-star of vine reviews = " + str(total_5star_Vine_paid)) 

The number of 5-star of vine reviews = 0


In [31]:
total_5star_Vine_not_paid = vine_not_paid_df.filter("star_rating = 5").count()
print ("The number of 5-star of non-Vine reviews = " + str(total_5star_Vine_not_paid))

The number of 5-star of non-Vine reviews = 22843


In [32]:
print ("The % of 5-star of vine reviews =  0 % ") # since total number is zero

The % of 5-star of vine reviews =  0 % 


In [33]:
percentage_5star_Vine_in_non_vine = round((total_5star_Vine_not_paid / total_number_of_vine_not_paid) * 100, 2)
print ("The number of 5-star of non-Vine reviews = " + str(percentage_5star_Vine_in_non_vine) + "%")

The number of 5-star of non-Vine reviews = 37.67%
